In [1]:
import warnings
warnings.filterwarnings("ignore")
from src.dataset import load_tokenized_data
from src.model import LyricsMelodyModel
from src.embeddings import extract_embedding_weights
from tensorflow.python.keras.callbacks import TensorBoard

Using TensorFlow backend.


In [2]:
tensorboard = TensorBoard(log_dir='./logs', histogram_freq=0, write_graph=True, write_images=False)                    

In [3]:
# x, y, tokenizer, songs = load_tokenized_data(with_melody=True, melody_type='doc2vec')
import joblib
x, y, tokenizer, songs = joblib.load('DOC2VEC_x_y_tokenizer_songs.jblib') 
embedding_matrix = extract_embedding_weights()

100%|███████████████████████████████| 614/614 [00:02<00:00, 273.13it/s]


In [4]:
model = LyricsMelodyModel(tokenizer, embedding_matrix, 
                          rnn_type='GRU', 
                          rnn_units=20, 
                          bidirectional=False,
                          dropout=0.4,
                          train_embedding=True, 
                          is_layer_norm=True)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 1, 300)       2247600     input_1[0][0]                    
__________________________________________________________________________________________________
flatten_1 (Flatten)             (None, 300)          0           embedding_1[0][0]                
__________________________________________________________________________________________________
input_2 (InputLayer) 

In [23]:
model.train([x,songs], y, epochs=20, batch_size=64, callbacks=[tensorboard])

Train on 171249 samples, validate on 19028 samples
Epoch 1/20
171249/171249 [==============================] - 21s 123us/step - loss: 5.6302 - _perplexity: 194.2186 - val_loss: 5.9617 - val__perplexity: 501.0435
Epoch 2/20
171249/171249 [==============================] - 18s 107us/step - loss: 5.3745 - _perplexity: 187.7750 - val_loss: 5.9315 - val__perplexity: 886.7739
Epoch 3/20
171249/171249 [==============================] - 18s 107us/step - loss: 5.2453 - _perplexity: 189.9077 - val_loss: 5.9549 - val__perplexity: 1744.5221
Epoch 4/20
171249/171249 [==============================] - 18s 106us/step - loss: 5.1702 - _perplexity: 193.6582 - val_loss: 5.9825 - val__perplexity: 3099.0460
Epoch 5/20
171249/171249 [==============================] - 18s 106us/step - loss: 5.1176 - _perplexity: 197.7953 - val_loss: 5.9678 - val__perplexity: 3938.3642
Epoch 6/20
171249/171249 [==============================] - 18s 107us/step - loss: 5.0817 - _perplexity: 201.6450 - val_loss: 5.9897 - val__p

In [48]:
from src.dataset import load_tokenized_test
test_data = load_tokenized_test(tokenizer, with_melody=True)
first_words, song_embeddings = zip(*[(song['X'][0], song['melody_embedding']) for song in test_data])

def parse_to_song(text):
    return text.replace('. ', '\n').replace(' \'', '\'').replace(' n\'t', 'n\'t').replace('eos', '')

def test_report():
    for i in range(5):
        curr = test_data[i]
        predicted_lyrics = model.predict(first_word=first_words[i], song=song_embeddings[i], n_words=250)
        lyrics = parse_to_song(predicted_lyrics)
        print(f"song_name: {curr['song_name']}\nartist: {curr['artist']}\nfirst word: {first_words[i]}\nPredicted song:\n{lyrics}\n\n")
        print("===========================================")
              
test_report()

In [49]:
# model.predict(first_word=first_words[1], song=song_embeddings[1], n_words=50)

In [50]:
test_report()

song_name: eternal flame
artist: the bangles
first word: close
Predicted song:
close to dear is gone 

you's in the so found letting have to get my eyes 
you never get on the playin microphone listen to wrapped out that's singing about it in me giving it's pain this again 
oh free 
i go through in right alone 
or can't think yeah 
people had their spring 
that the now that you're gon na dance 
if i see where it bomber 
you like friends are not emotion in town 
if you 
put away 
no ooh 
i'm not no can you faith 
that's so flames are in your words brother 
i only tears i still the loneliness up the sign 
 
yeah is darling the life 
getting boy to make it's free lord and i can have 
spirit your follow you can dream laugh i'll pum pum pum about me 
sometimes i let can say allentown 
when i just silver up in that but i'm a bells people are 
uh the dove your movie 
you will be brought out the world 
do 
so says well you 
far too good always out of this timing 
i'm a sound 
learn 
great holdi